# **Módulo 3 - Manipulando dados com Spark - Parte I**

## **Importando bibliotecas do Spark**

In [1]:
%%bash

# Instal Java
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

**OBS :** A preparação do ambiente não é igual ao da aula porque estou fazendo 
o exercício no Google Colab. 

In [ ]:
df = spark.createDataFrame([('Fulano','1'),
                            ('Ciclano','2')], 
                           schema='nome STRING, id STRING')

In [ ]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
| Fulano|  1|
|Ciclano|  2|
+-------+---+



## **Acessando os Tipos do Spark**

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import col

In [ ]:
int_type = IntegerType()

In [ ]:
array_type = ArrayType(IntegerType())

In [ ]:
array_type

ArrayType(IntegerType,true)

## **Convertendo os tipos de Colunas**

In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'string')]

In [ ]:
df.select('nome',col('id').cast(IntegerType()))

DataFrame[nome: string, id: int]

In [ ]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
| Fulano|  1|
|Ciclano|  2|
+-------+---+



In [ ]:
df.select('nome',col('id').cast('int'))

DataFrame[nome: string, id: int]

In [ ]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
| Fulano|  1|
|Ciclano|  2|
+-------+---+



## **Schema e Criação de DataFrames**

Um schema no Spark é uma especificação de tipos de colunas de um DataFrame. Eles são usadaos na leitura de dados externos e criação de DataFrames, e podem
ser passados diretamente no Spark ou podem ser inferidos. Passar um schema na 
leitura traz benefícios interessantes como : 

  - Evita que o Spark faça inferência de tipos, o que é custoso e demorado 
  dependendo do tamanho do arquivo, laém de propenso a erros; 
  - Permite que usuário identifique erros nos dados logo na leitura, caso 
  não sigam o schema especificado. 

In [ ]:
df = spark.createDataFrame([('Fulano'  ,1),
                            ('Ciclano' ,2),
                            ('Beltrano',3),
                            ('Deltrano',4)], 
                            schema=['nome','id'])

In [ ]:
df.show()

+--------+---+
|    nome| id|
+--------+---+
|  Fulano|  1|
| Ciclano|  2|
|Beltrano|  3|
|Deltrano|  4|
+--------+---+



In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'bigint')]

## **Criando schemas programaticamente**

In [ ]:
schema = \
  StructType([
    StructField('nome',StringType()),
    StructField('id'  ,IntegerType())
  ])

In [ ]:
df = spark.createDataFrame([('Fulano'  ,1),
                            ('Ciclano' ,2),
                            ('Beltrano',3),
                            ('Deltrano',4)], 
                            schema=schema)

In [ ]:
df.show()

+--------+---+
|    nome| id|
+--------+---+
|  Fulano|  1|
| Ciclano|  2|
|Beltrano|  3|
|Deltrano|  4|
+--------+---+



In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## **Criando schemas com DDL**

In [ ]:
schema = 'nome STRING, id INT' 

In [ ]:
df = spark.createDataFrame([('Fulano'  ,1),
                            ('Ciclano' ,2),
                            ('Beltrano',3),
                            ('Deltrano',4)], 
                            schema=schema)

In [ ]:
df.show()

+--------+---+
|    nome| id|
+--------+---+
|  Fulano|  1|
| Ciclano|  2|
|Beltrano|  3|
|Deltrano|  4|
+--------+---+



In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## **Criando DataFrames**

In [ ]:
data = [('Fulano'  ,1),
        ('Ciclano' ,2),
        ('Beltrano',3),
        ('Deltrano',4)]

In [ ]:
schema = 'nome STRING, id INT' 

In [ ]:
df = spark.createDataFrame(data,schema=schema)

In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

In [ ]:
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- id: integer (nullable = true)



In [ ]:
spark.range(100).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



## **Leitura e Escrita de Dados**

In [ ]:
link_tab = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/tab_cnae.csv'

### **DataFrameReader**

      `spark.read.format(format).option(args).load(file_path)`

### **DataFrameWriter**

      `spark.write.format(format).option(args).load(file_path)`

### **Lendo e Escrevendo CSV**
Opções mais comuns : 
  - header
  - inferSchema
  - sep
  - encoding

In [ ]:
#df = spark.read.format('csv').load(link_tab)
df = spark.read.csv(link_tab,sep=',',header=True)

In [ ]:
df.limit(5).show()

+------+--------------------+---------+-----------+
|  CNAE|           DESCRIÇÃO|CÓD.SETOR| NOME SETOR|
+------+--------------------+---------+-----------+
|111301|    Cultivo de arroz|        1|AGRICULTURA|
|111302|    Cultivo de milho|        1|AGRICULTURA|
|111303|    Cultivo de trigo|        1|AGRICULTURA|
|111399|Cultivo de outros...|        1|AGRICULTURA|
|112101|Cultivo de algodã...|        1|AGRICULTURA|
+------+--------------------+---------+-----------+



## **Definindo o schema**

In [ ]:
schema = 'cod_cnae STRING, descricao STRING, cod_setor INT, nome_setor STRING '

In [ ]:
df = spark.read.csv(link_tab,sep=',',header=True, schema=schema)

In [ ]:
df.dtypes

[('cod_cnae', 'string'),
 ('descricao', 'string'),
 ('cod_setor', 'int'),
 ('nome_setor', 'string')]

outra forma de fazer seria

In [ ]:
df = (
    spark.read
    .format('csv')
    .option('header','true')
    .option('sep',',')
    .schema(schema)
    .load(link_tab)
    )

df.limit(5).show()

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+



In [ ]:
df = (
    spark.read
    .format('csv')
    .options(header=True,sep=',')  
    .schema(schema)
    .load(link_tab)
)
df.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



**OBS :** Utilizando o método "options" podemos parametrizar melhor nossa função
 usando um dicionário

In [ ]:
options_dict = {
    'sep' : ',' , 
    'header' : 'True'
}

df = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .schema(schema)
    .load(link_tab)
)
df.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- cod_cnae: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



In [ ]:
schema = 'cod_cnae INT, descricao STRING, cod_setor INT, nome_setor STRING '

In [ ]:
options_dict = {
    'sep' : ',' , 
    'header' : 'True'
}

df = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .schema(schema)
    .load(link_tab)
)
df.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



In [ ]:
path_save = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/'

In [ ]:
#df.write.format('csv').save(path_save + 'df_cnae_teste',header=True)

In [ ]:
spark.read.format('csv').load(path_save + 'df_cnae_teste',header=True).printSchema()

root
 |-- cod_cnae: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: string (nullable = true)
 |-- nome_setor: string (nullable = true)



In [ ]:
df.toPandas().to_csv(
    path_save + 'df_cnae_teste.csv',
    index=False, header=True
    )

# **Aula 3.4.2 -** Leitura e escrita de dados - Parte II

## **Lendo e Escrevendo JSON**

In [ ]:
df.write.format('json') \
  .mode('overwrite')  \
  .save(path_save + 'df_cnae_teste.json')

In [ ]:
df_json = spark.read.format('json') \
  .load(path_save + 'df_cnae_teste.json')

In [ ]:
df_json.show()

+--------+---------+--------------------+-----------+
|cod_cnae|cod_setor|           descricao| nome_setor|
+--------+---------+--------------------+-----------+
|  111301|        1|    Cultivo de arroz|AGRICULTURA|
|  111302|        1|    Cultivo de milho|AGRICULTURA|
|  111303|        1|    Cultivo de trigo|AGRICULTURA|
|  111399|        1|Cultivo de outros...|AGRICULTURA|
|  112101|        1|Cultivo de algodã...|AGRICULTURA|
|  112102|        1|     Cultivo de juta|AGRICULTURA|
|  112199|        1|Cultivo de outras...|AGRICULTURA|
|  113000|        1|Cultivo de cana-d...|AGRICULTURA|
|  114800|        1|     Cultivo de fumo|AGRICULTURA|
|  115600|        1|     Cultivo de soja|AGRICULTURA|
|  116401|        1| Cultivo de amendoim|AGRICULTURA|
|  116402|        1| Cultivo de girassol|AGRICULTURA|
|  116403|        1|   Cultivo de mamona|AGRICULTURA|
|  116499|        1|Cultivo de outras...|AGRICULTURA|
|  119901|        1|  Cultivo de abacaxi|AGRICULTURA|
|  119902|        1|     Cul

In [ ]:
df_json.printSchema()

root
 |-- cod_cnae: long (nullable = true)
 |-- cod_setor: long (nullable = true)
 |-- descricao: string (nullable = true)
 |-- nome_setor: string (nullable = true)



## **Lendo e Escrevendo ORC**



In [ ]:
df.write.format('orc') \
  .mode('overwrite') \
  .save(path_save + 'df_cnae_teste.orc')

In [ ]:
df_orc = spark.read.format('orc').load(path_save + 'df_cnae_teste.orc')

In [ ]:
df_orc.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df_orc.printSchema()

root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



## **Lendo e Escrevendo Parquet**

  - Armazenamento colunar, em contraste com o CSV, que armazena baseado nas linhas. Assim, quando uma query é realizada é possível ignorar os dados não relevantes de maneira rápida e fácil, resultando em operações bem mais eficientes; 
  - Preservação de metadados, incluindo os tipos das colunas, o que garante eficiência e praticidade na escrita e leitura (não é necessário especificar schemas para arquivos parquet);
  - Suporte a dados estruturados de forma aninhada, como listas;
  - Otimizado para processar dados particionados com volume na casa dos gigabytes para cada arquivo;
  - Compressão de dados na escrita, de forma a ocupar menos espaço;
  - Integração com ferramentas como AWS athena, Amazon Redshift Spectrum, Google Big Query e Google Dataproc. 

In [ ]:
df.write.format('parquet') \
  .mode('overwrite') \
  .save(path_save + 'df_cnae_teste.pq')

In [ ]:
df_parquet = spark.read.format('parquet').load(path_save + 'df_cnae_teste.pq')

In [ ]:
df_parquet.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df_parquet.printSchema()

root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



mode : 
  
  - append : arquivos empilhados aos já existentes
  - ignore : retorna um erro silencioso
  - overwrite : sobrescreve os dados já exsitentes
  -  error (default) : retorne erro se já existem dados

In [ ]:
df.write.format('parquet').save(path_save + 'df_cnae_teste.pq', 
                                mode='overwrite'
                                )

ou

In [ ]:
df.write.format('parquet').mode('overwrite').save(path_save + 'df_cnae_teste.pq')

# **Aula 3.5.1 -** Operações básicas com DataFrames - Parte I

## **Manipulação dos Dados**

In [4]:
imdb_path = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_basics.csv'

In [5]:
options_dict = {
    'sep' : '\t' , 
    'header' : 'True'
}

df_titles = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .load(imdb_path)
)

df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

## **Colunas e Expressões**

As colunas são a principal unidade de manipulação de dados do Spark

In [6]:
from pyspark.sql.functions import col,round

In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',col('runTimeMinutes').cast('int')/60)
    .show(5)
)

+---------+--------------------+--------------+--------------------+
|   tconst|        primaryTitle|runtimeMinutes|        runtimeHours|
+---------+--------------------+--------------+--------------------+
|tt0000001|          Carmencita|             1|0.016666666666666666|
|tt0000002|Le clown et ses c...|             5| 0.08333333333333333|
|tt0000003|      Pauvre Pierrot|             4| 0.06666666666666667|
|tt0000004|         Un bon bock|            12|                 0.2|
|tt0000005|    Blacksmith Scene|             1|0.016666666666666666|
+---------+--------------------+--------------+--------------------+
only showing top 5 rows



In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',round(col('runTimeMinutes').cast('int')/60,3))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



In [ ]:
df_titles['runtimeMinutes']

Column<'runtimeMinutes'>

Forma "pandas" de selecionar

  1. df.coluna
  2. df['coluna']

In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',round(col('runTimeMinutes').cast('int')/60,3))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',df_titles['runtimeMinutes'].cast('int')/60)
    #.withColumn('hour_plus2',df_titles['runtimeHours'] + 2)
    .show(5)
)

+---------+--------------------+--------------+--------------------+
|   tconst|        primaryTitle|runtimeMinutes|        runtimeHours|
+---------+--------------------+--------------+--------------------+
|tt0000001|          Carmencita|             1|0.016666666666666666|
|tt0000002|Le clown et ses c...|             5| 0.08333333333333333|
|tt0000003|      Pauvre Pierrot|             4| 0.06666666666666667|
|tt0000004|         Un bon bock|            12|                 0.2|
|tt0000005|    Blacksmith Scene|             1|0.016666666666666666|
+---------+--------------------+--------------+--------------------+
only showing top 5 rows



In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',col('runtimeMinutes').cast('int')/60)
    .withColumn('hours_plus2' ,col('runtimeHours') + 2)
    .show(5)
)

+---------+--------------------+--------------+--------------------+------------------+
|   tconst|        primaryTitle|runtimeMinutes|        runtimeHours|       hours_plus2|
+---------+--------------------+--------------+--------------------+------------------+
|tt0000001|          Carmencita|             1|0.016666666666666666|2.0166666666666666|
|tt0000002|Le clown et ses c...|             5| 0.08333333333333333|2.0833333333333335|
|tt0000003|      Pauvre Pierrot|             4| 0.06666666666666667| 2.066666666666667|
|tt0000004|         Un bon bock|            12|                 0.2|               2.2|
|tt0000005|    Blacksmith Scene|             1|0.016666666666666666|2.0166666666666666|
+---------+--------------------+--------------+--------------------+------------------+
only showing top 5 rows



### **Expressões**

In [7]:
from pyspark.sql.functions import expr

In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',expr('cast(runtimeMinutes as INT)/60'))
    .show(5)
)

+---------+--------------------+--------------+--------------------+
|   tconst|        primaryTitle|runtimeMinutes|        runtimeHours|
+---------+--------------------+--------------+--------------------+
|tt0000001|          Carmencita|             1|0.016666666666666666|
|tt0000002|Le clown et ses c...|             5| 0.08333333333333333|
|tt0000003|      Pauvre Pierrot|             4| 0.06666666666666667|
|tt0000004|         Un bon bock|            12|                 0.2|
|tt0000005|    Blacksmith Scene|             1|0.016666666666666666|
+---------+--------------------+--------------+--------------------+
only showing top 5 rows



In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',expr('round(cast(runtimeMinutes as INT)/60,3)'))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



# **Aula 3.5.2 -** Operações básicas com DataFrames - Parte II

## **Manipulação dos Dados**

In [ ]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
df_titles.select('tconst','primaryTitle','genres').show(5)

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|              genres|
+---------+--------------------+--------------------+
|tt0000001|          Carmencita|   Documentary,Short|
|tt0000002|Le clown et ses c...|     Animation,Short|
|tt0000003|      Pauvre Pierrot|Animation,Comedy,...|
|tt0000004|         Un bon bock|     Animation,Short|
|tt0000005|    Blacksmith Scene|        Comedy,Short|
+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df_titles.columns

['tconst',
 'titleType',
 'primaryTitle',
 'originalTitle',
 'isAdult',
 'startYear',
 'endYear',
 'runtimeMinutes',
 'genres']

In [ ]:
select_cols = [c for c in df_titles.columns if c.find('Title') != -1]

In [ ]:
df_titles.select(select_cols).show(5)

+--------------------+--------------------+
|        primaryTitle|       originalTitle|
+--------------------+--------------------+
|          Carmencita|          Carmencita|
|Le clown et ses c...|Le clown et ses c...|
|      Pauvre Pierrot|      Pauvre Pierrot|
|         Un bon bock|         Un bon bock|
|    Blacksmith Scene|    Blacksmith Scene|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
cols = ['tconst','primaryTitle','genres']
df_titles.select(cols).show(10)

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|              genres|
+---------+--------------------+--------------------+
|tt0000001|          Carmencita|   Documentary,Short|
|tt0000002|Le clown et ses c...|     Animation,Short|
|tt0000003|      Pauvre Pierrot|Animation,Comedy,...|
|tt0000004|         Un bon bock|     Animation,Short|
|tt0000005|    Blacksmith Scene|        Comedy,Short|
|tt0000006|   Chinese Opium Den|               Short|
|tt0000007|Corbett and Court...|         Short,Sport|
|tt0000008|Edison Kinetoscop...|   Documentary,Short|
|tt0000009|          Miss Jerry|       Romance,Short|
|tt0000010| Leaving the Factory|   Documentary,Short|
+---------+--------------------+--------------------+
only showing top 10 rows



In [ ]:
cols = ['primaryTitle','genres']
df_titles.select('tconst',*cols).show(5)

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|              genres|
+---------+--------------------+--------------------+
|tt0000001|          Carmencita|   Documentary,Short|
|tt0000002|Le clown et ses c...|     Animation,Short|
|tt0000003|      Pauvre Pierrot|Animation,Comedy,...|
|tt0000004|         Un bon bock|     Animation,Short|
|tt0000005|    Blacksmith Scene|        Comedy,Short|
+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df_titles.select('*').show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

**Observações :**

  - Podemos realizar operações sobre colunas selecionadas
  - A ordem em que as colunas são selecionadas é a ordem em que elas vão ser inseridas no DataFrame resultante.

In [ ]:
from pyspark.sql.functions import upper

df_titles.select('tconst',
                 'genres',
                 upper('primaryTitle').alias('primaryTitle')).show(5)

+---------+--------------------+--------------------+
|   tconst|              genres|        primaryTitle|
+---------+--------------------+--------------------+
|tt0000001|   Documentary,Short|          CARMENCITA|
|tt0000002|     Animation,Short|LE CLOWN ET SES C...|
|tt0000003|Animation,Comedy,...|      PAUVRE PIERROT|
|tt0000004|     Animation,Short|         UN BON BOCK|
|tt0000005|        Comedy,Short|    BLACKSMITH SCENE|
+---------+--------------------+--------------------+
only showing top 5 rows



**Selecionando valores distintos**

In [ ]:
df_titles.select('startYear').distinct().show()

+---------+
|startYear|
+---------+
|     1903|
|     1953|
|     1897|
|     1957|
|     1987|
|     1956|
|     1936|
|     2016|
|     2020|
|     2012|
|     1958|
|     1910|
|     1943|
|     1915|
|     1972|
|     1931|
|     1911|
|     1926|
|     1938|
|     1988|
+---------+
only showing top 20 rows



In [ ]:
df_titles.distinct().show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000033|    short|  Horse Trick Riders|          La voltige|      0|     1895|     \N|             1|Comedy,Documentar...|
|tt0000132|    short|          Card Party|Une partie de cartes|      0|     1896|     \N|             1|     Biography,Short|
|tt0214902|    short|        The Magician|         Le magicien|      0|     1898|     \N|             1|Fantasy,Horror,Short|
|tt0225248|    short|Canada Vignettes:...|Canada Vignettes:...|      0|     1979|     \N|             1|     Animation,Short|
|tt0245776|    short|       The Biter Bit|       The Biter Bit|      0|     1899|     \N|             1|        Comedy

In [ ]:
df_titles.count()

8328316

In [ ]:
df_titles.distinct().explain('formatted')

== Physical Plan ==
AdaptiveSparkPlan (5)
+- HashAggregate (4)
   +- Exchange (3)
      +- HashAggregate (2)
         +- Scan csv  (1)


(1) Scan csv 
Output [9]: [tconst#16, titleType#17, primaryTitle#18, originalTitle#19, isAdult#20, startYear#21, endYear#22, runtimeMinutes#23, genres#24]
Batched: false
Location: InMemoryFileIndex [file:/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_basics.csv]
ReadSchema: struct<tconst:string,titleType:string,primaryTitle:string,originalTitle:string,isAdult:string,startYear:string,endYear:string,runtimeMinutes:string,genres:string>

(2) HashAggregate
Input [9]: [tconst#16, titleType#17, primaryTitle#18, originalTitle#19, isAdult#20, startYear#21, endYear#22, runtimeMinutes#23, genres#24]
Keys [9]: [runtimeMinutes#23, tconst#16, titleType#17, originalTitle#19, startYear#21, endYear#22, primaryTitle#18, isAdult#20, genres#24]
Functions: []
Aggregate Attributes: []
Results [9]: [runtimeMinutes#23, tconst#16, titleType#17, originalTitl

**Filtros**

Operadores lógicos : 
   - e   : &
   - ou  : |
   - não : ~

Para fazer o filtro, pode ser usado tanto a função `filter()` como `where()` 

**Filtros com uma condição**

In [ ]:
(
    df_titles.filter(col('titleType') == 'movie').show(5)
)

+---------+---------+------------------+------------------+-------+---------+-------+--------------+------+
|   tconst|titleType|      primaryTitle|     originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|
+---------+---------+------------------+------------------+-------+---------+-------+--------------+------+
|tt0000502|    movie|          Bohemios|          Bohemios|      0|     1905|     \N|           100|    \N|
|tt0000591|    movie|  The Prodigal Son| L'enfant prodigue|      0|     1907|     \N|            90| Drama|
|tt0000615|    movie|Robbery Under Arms|Robbery Under Arms|      0|     1907|     \N|            \N| Drama|
|tt0000630|    movie|            Hamlet|            Amleto|      0|     1908|     \N|            \N| Drama|
|tt0000675|    movie|       Don Quijote|       Don Quijote|      0|     1908|     \N|            \N| Drama|
+---------+---------+------------------+------------------+-------+---------+-------+--------------+------+
only showing top 5 rows



In [ ]:
(
    df_titles.where(col('titleType') == 'movie').show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|               \N|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|            Drama|
|tt0000615|    movie|  Robbery Under Arms|  Robbery Under Arms|      0|     1907|     \N|            \N|            Drama|
|tt0000630|    movie|              Hamlet|              Amleto|      0|     1908|     \N|            \N|            Drama|
|tt0000675|    movie|         Don Quijote|         Don Quijote|      0|     1908|     \N|            \N|            Drama|
|tt0000679|    m

In [ ]:
(
    df_titles.where(col('titleType') == 'movie').count()
)

591977

In [ ]:
(
    df_titles.filter((col('titleType') == 'movie') & (col('runTimeMinutes') <= 90)).show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
    df_titles.filter(((col('titleType') == 'movie') | (col('titleType') == 'tvSeries')) & (col('runtimeMinutes') <= 90)).show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
df_titles.filter( ((col('titleType') == 'movie') & (col('titleType') == 'tvSeries') & (col('runtimeMinutes') <= 90)) ).count()
)

0

In [ ]:
(
    df_titles.filter(((col('titleType') == 'movie') | (col('titleType') == 'tvSeries')) & (col('runtimeMinutes') <= 90)).count()
)

296182

In [ ]:
df_titles.filter((col('titleType').isin('movie','tvSeries')) & (col('runtimeMinutes') <= 90)).show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
df_titles
.filter(col('titleType').isin('movie','tvSeries'))
.filter(col('runtimeMinutes') <= 90)
.count()
)

296182

**Filtros utilizando expressões**

In [ ]:
(
    df_titles
    .filter('titleType == "movie"')
    .filter('runtimeMinutes <= 90')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
    df_titles
    .filter('titleType in ("movie","tvSeries") and runtimeMinutes <= 90')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

**Observações**

Quando nos referimos às colunas por meio da função `col()`, temos acesso à diversos métodos das colunas que podem ser utilizados para auxiliar na filtragem do DataFrame. Alguns deles são :

  - `isin()` : checa se a coluna contém os valores listados na função.
  - `contains` : utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Aceita uma outra coluna de text.
  - `like()` : utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Funciona de forma similar so "LIKE" do SQL.
  - `rlike()` : utilizado para verificar se uma coluna de texto contém algum padrão especificado (**aceita regex**). Funciona de forma similar ao "RLIKE" do SQL
  - `startswith()` : utilizado para verificar se uma coluna de texto começa com algum padrão especificado (**aceita regex**).
  - `endswith()` :  utilizado para verificar se uma coluna de texto termina com algum padrão especificado (**aceita regex**).
  - `between()` : checa se os valores da coluna estão dentro do intervalo especificado. Os dois lados do intervalo são inclusivos
  - `isNull()` : retorna True se o valor da coluna é nulo
  - `isNotNull()` : retorna True se o valor da coluna é não nulo

Outros métodos úteis :

  - `alias()/name()` : usado para renomear as colunas em operações como select() e agg()
  - `astype()/cast()` : usado para mudar o tipo das colunas. Aceita tanto um
  string como um tipo especificado pelo módulo pyspark.sql.types
  - `substr()` : utilizado para cortar uma string com base em índices dos caracteres.  

In [ ]:
(
  df_titles
 .filter(col('primaryTitle').like('%Avengers%'))
 .filter(col('titleType') == 'movie')
 .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0034639|    movie|        The Avengers|   The Day Will Dawn|      0|     1942|     \N|            98|           Drama,War|
|tt0036194|    movie|The People's Aven...|   Narodnye mstiteli|      0|     1943|     \N|            55|     Documentary,War|
|tt0058651|    movie|  The Three Avengers| Gli invincibili tre|      0|     1964|     \N|           101|Action,Adventure,...|
|tt0069746|    movie|Avengers of the Reef|Avengers of the Reef|      0|     1973|     \N|            84|    Adventure,Family|
|tt0074513|    movie|The Shaolin Avengers|Fang Shi Yu yu Hu...|      0|     1976|     \N|            97|        Action

In [ ]:
(
    df_titles.
    withColumn('startYear',col('startYear').cast('int'))
    .filter(col('startYear').isNotNull())
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

# **Aula 3.5.3 -** Operações básicas com DataFrames - Parte III

**Ordenando o DataFrame**

A ordenação do DataFrame pode ser feita utilizando funções `orderBy()` ou `sort()`. Algumas funções auxiliares importantes para serem usadas ao ordenar : 

  - `asc()`: ordena a coluna de forma ascendente (default)
  - `desc()` : ordena a coluna de forma descrescente
  - `asc_nulls_first()/desc_nulls_first()`: ordena a coluna de forma ascendente e descendente, respectivamente, mantendo os campos nulos primeiro
  - `asc_nulls_last()/desc_nulls_last()`: ordena a coluna de forma ascendente e
  decrescente, respectivamente, mantendo os campos nulos por último

In [ ]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
df_titles.orderBy(col('startYear').cast('int')).show()

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|       genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-------------+
|tt11112784|    movie|   Perfect Strangers|   Perfect Strangers|      0|       \N|     \N|            \N| Comedy,Drama|
|tt11113108|tvEpisode|      Episode #1.759|      Episode #1.759|      0|       \N|     \N|            \N|Comedy,Family|
|tt11112532|    movie|                Agra|                Agra|      0|       \N|     \N|            \N|        Drama|
|tt11113062|tvEpisode|      Episode #1.741|      Episode #1.741|      0|       \N|     \N|            \N|Comedy,Family|
|tt11112858|    movie|Lizards: A Pop Opera|Lizards: A Pop Opera|      0|       \N|     \N|            \N|        Drama|
|tt11113066|tvEpisode|      Episode #1.7

In [ ]:
df_titles.orderBy('startYear','runtimeMinutes').filter('titleType == "movie"').show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt2210499|    movie|          Birmingham|          Birmingham|      0|     1896|     \N|            61|         Documentary|
|tt0229676|    movie|Reproduction of t...|Reproduction of t...|      0|     1897|     \N|            \N|Documentary,News,...|
|tt0236940|    movie|69th Regiment Pas...|69th Regiment Pas...|      0|     1898|     \N|            \N|         Documentary|
|tt0235357|    movie|Dressing Paper Dolls|Dressing Paper Dolls|      0|     1898|     \N|            \N|         Documentary|
|tt0138774|    movie|Saída dos Operári...|Saída dos Operári...|      0|     1898|     \N|            \N|         Docum

In [ ]:
from pyspark.sql.functions import desc

df_titles.orderBy('startYear',desc('runtimeMinutes')).filter('titleType == "movie"').show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt2210499|    movie|          Birmingham|          Birmingham|      0|     1896|     \N|            61|         Documentary|
|tt0229676|    movie|Reproduction of t...|Reproduction of t...|      0|     1897|     \N|            \N|Documentary,News,...|
|tt0236940|    movie|69th Regiment Pas...|69th Regiment Pas...|      0|     1898|     \N|            \N|         Documentary|
|tt0235357|    movie|Dressing Paper Dolls|Dressing Paper Dolls|      0|     1898|     \N|            \N|         Documentary|
|tt0138349|    movie|O Carnaval em Lisboa|O Carnaval em Lisboa|      0|     1898|     \N|            \N|         Docum

In [ ]:
df_titles.orderBy(desc('startYear')).filter('titleType == "movie"').show()

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt11115868|    movie|        The Shinobis|        The Shinobis|      0|       \N|     \N|            \N|              Action|
|tt11116102|    movie|              Deeper|              Deeper|      0|       \N|     \N|            \N|             Romance|
|tt11116112|    movie|Untitled Elliott ...|Untitled Elliott ...|      0|       \N|     \N|            \N|                  \N|
|tt11112784|    movie|   Perfect Strangers|   Perfect Strangers|      0|       \N|     \N|            \N|        Comedy,Drama|
|tt11116440|    movie|               Shame|               Shame|      0|       \N|     \N|            \N|      

In [ ]:
from pyspark.sql.functions import desc

(
 df_titles.withColumn('startYear' , col('startYear').cast('int'))
 .orderBy('startYear')
 .filter('titleType == "movie"')
 .show()
)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt11116440|    movie|               Shame|               Shame|      0|     null|     \N|            \N|                  \N|
|tt11122910|    movie|       Durgo Rahasya|       Durgo Rahasya|      0|     null|     \N|            \N|      Drama,Thriller|
|tt11116516|    movie|        Spiritwalker|        Spiritwalker|      0|     null|     \N|            \N|      Fantasy,Horror|
|tt11112072|    movie|       Hell Can Wait|       Hell Can Wait|      0|     null|     \N|            \N|              Comedy|
|tt11116536|    movie|           3rd Floor|           3rd Floor|      0|     null|     \N|            \N|      

In [ ]:
from pyspark.sql.functions import asc_nulls_first

(
  df_titles
 .withColumn('startYear',col('startYear').cast('int'))
 .orderBy(asc_nulls_first('startYear'))
 .show(5)   
)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt11112072|    movie|       Hell Can Wait|       Hell Can Wait|      0|     null|     \N|            \N|           Comedy|
|tt11112078|    movie|  Un día 28 de enero|  Un día 28 de enero|      0|     null|     \N|            \N|      Documentary|
|tt11112202|    movie|                Kajo|                Kajo|      0|     null|     \N|            \N|Adventure,Romance|
|tt11112084|    short|Salt. Pepper. Pea...|Salt. Pepper. Pea...|      0|     null|     \N|            \N|      Crime,Short|
|tt11112002|  tvMovie|   The most Precious|   The most Precious|      0|     null|     \N|            \N|            Drama|
+-------

**Renomeando Colunas**

Para renomear colunas, é utilizada a função `withColumnRename()` , da seguinte forma

  `df.withColumnRenamed("nome_antigo","nome_novo")`

In [ ]:
(
    df_titles
    .withColumnRenamed('primaryTitle','nome_filme')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|          nome_filme|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
(
    df_titles
    .withColumnRenamed('primaryTitle','nome_filme')
    .select('nome_filme','titleType','startYear','runtimeMinutes')
    .show()
)

+--------------------+---------+---------+--------------+
|          nome_filme|titleType|startYear|runtimeMinutes|
+--------------------+---------+---------+--------------+
|          Carmencita|    short|     1894|             1|
|Le clown et ses c...|    short|     1892|             5|
|      Pauvre Pierrot|    short|     1892|             4|
|         Un bon bock|    short|     1892|            12|
|    Blacksmith Scene|    short|     1893|             1|
|   Chinese Opium Den|    short|     1894|             1|
|Corbett and Court...|    short|     1894|             1|
|Edison Kinetoscop...|    short|     1894|             1|
|          Miss Jerry|    short|     1894|            40|
| Leaving the Factory|    short|     1895|             1|
|Akrobatisches Pot...|    short|     1895|             1|
|The Arrival of a ...|    short|     1896|             1|
|The Photographica...|    short|     1895|             1|
| The Waterer Watered|    short|     1895|             1|
| Autour d'une

In [ ]:
(
    df_titles.selectExpr('primaryTitle as nome_filme', 'titleType', 
                         'startYear','runtimeMinutes').show(5)
)

+--------------------+---------+---------+--------------+
|          nome_filme|titleType|startYear|runtimeMinutes|
+--------------------+---------+---------+--------------+
|          Carmencita|    short|     1894|             1|
|Le clown et ses c...|    short|     1892|             5|
|      Pauvre Pierrot|    short|     1892|             4|
|         Un bon bock|    short|     1892|            12|
|    Blacksmith Scene|    short|     1893|             1|
+--------------------+---------+---------+--------------+
only showing top 5 rows



In [ ]:
df_renamed = df_titles
for c in df_titles.columns : 
  df_renamed = df_renamed.withColumnRenamed(c,c+'_suffx')

df_renamed.limit(5).toPandas()

,tconst_suffx,titleType_suffx,primaryTitle_suffx,originalTitle_suffx,isAdult_suffx,startYear_suffx,endYear_suffx,runtimeMinutes_suffx,genres_suffx
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


**Criando e Alterando Colunas**

Para criar ou alterar colunas é utilizada a função `withColumn()`, da seguinte forma : 
  - `df.withColumn("nome_da_coluna", {expressão geradora da coluna})`

In [ ]:
from pyspark.sql.functions import upper

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn("primaryTitle_2",upper('primaryTitle'))
    .show(5)
)

+---------+--------------------+--------------+--------------------+
|   tconst|        primaryTitle|runtimeMinutes|      primaryTitle_2|
+---------+--------------------+--------------+--------------------+
|tt0000001|          Carmencita|             1|          CARMENCITA|
|tt0000002|Le clown et ses c...|             5|LE CLOWN ET SES C...|
|tt0000003|      Pauvre Pierrot|             4|      PAUVRE PIERROT|
|tt0000004|         Un bon bock|            12|         UN BON BOCK|
|tt0000005|    Blacksmith Scene|             1|    BLACKSMITH SCENE|
+---------+--------------------+--------------+--------------------+
only showing top 5 rows



**Criando colunas a partir de constantes**

In [ ]:
from pyspark.sql.functions import lit

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('pais',lit('Brasil'))
    .show(5)
)

+---------+--------------------+--------------+------+
|   tconst|        primaryTitle|runtimeMinutes|  pais|
+---------+--------------------+--------------+------+
|tt0000001|          Carmencita|             1|Brasil|
|tt0000002|Le clown et ses c...|             5|Brasil|
|tt0000003|      Pauvre Pierrot|             4|Brasil|
|tt0000004|         Un bon bock|            12|Brasil|
|tt0000005|    Blacksmith Scene|             1|Brasil|
+---------+--------------------+--------------+------+
only showing top 5 rows



**Criando colunas condicionais**

In [ ]:
from pyspark.sql.functions import when

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeMinutes',col('runtimeMinutes').cast('int'))
    .withColumn("categoria_runtime", when(col('runtimeMinutes') <=60,'curto')
                                    .when((col('runtimeMinutes') > 60) & (col('runtimeMinutes') < 120),'normal')
                                    .when(col('runtimeMinutes') >=120, 'longo')
                                    .when(col('runtimeMinutes').isNull(), 'nulo')
                                    .otherwise('Erro'))
    .filter('runtimeMinutes > 60')
    .show(25)
)

+---------+--------------------+--------------+-----------------+
|   tconst|        primaryTitle|runtimeMinutes|categoria_runtime|
+---------+--------------------+--------------+-----------------+
|tt0000502|            Bohemios|           100|           normal|
|tt0000574|The Story of the ...|            70|           normal|
|tt0000591|    The Prodigal Son|            90|           normal|
|tt0000679|The Fairylogue an...|           120|            longo|
|tt0001756|Lucha por la here...|            92|           normal|
|tt0002026|Anny - Story of a...|            68|           normal|
|tt0002101|           Cleopatra|           100|           normal|
|tt0002130|     Dante's Inferno|            71|           normal|
|tt0002315|El lobo de la sierra|            76|           normal|
|tt0002423|             Passion|            85|           normal|
|tt0002445|          Quo Vadis?|           120|            longo|
|tt0002452|The Independence ...|           120|            longo|
|tt0002625

In [ ]:
from pyspark.sql.functions import when,expr

predicado = """

CASE WHEN runtimeMinutes <= 60 THEN 'curto'
     WHEN runtimeMinutes > 60 AND runtimeMinutes < 120 THEN 'normal'
     WHEN runtimeMinutes >= 120 THEN 'longo'
     WHEN runtimeMinutes IS NULL THEN 'nulo'
     ELSE 'Erro'
END

"""

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeMinutes',col('runtimeMinutes').cast('int'))
    .withColumn("categoria_runtime", expr(predicado))
    .filter('runtimeMinutes > 60')
    .show(25)
)

+---------+--------------------+--------------+-----------------+
|   tconst|        primaryTitle|runtimeMinutes|categoria_runtime|
+---------+--------------------+--------------+-----------------+
|tt0000502|            Bohemios|           100|           normal|
|tt0000574|The Story of the ...|            70|           normal|
|tt0000591|    The Prodigal Son|            90|           normal|
|tt0000679|The Fairylogue an...|           120|            longo|
|tt0001756|Lucha por la here...|            92|           normal|
|tt0002026|Anny - Story of a...|            68|           normal|
|tt0002101|           Cleopatra|           100|           normal|
|tt0002130|     Dante's Inferno|            71|           normal|
|tt0002315|El lobo de la sierra|            76|           normal|
|tt0002423|             Passion|            85|           normal|
|tt0002445|          Quo Vadis?|           120|            longo|
|tt0002452|The Independence ...|           120|            longo|
|tt0002625

# **Aula 3.6 -** Trabalhando com diferentes tipos de dados


In [8]:
import pyspark.sql.functions as f

**Valores Numéricos**

  - `round()` : arredonda o valor numérico
  - `ceil` : arredonda o valor numérico para o maior inteiro mais próximo
  - `floor()` : arredonda o valure numérico para o menor inteiro mais próximo
  - `sqrt()` : retorna a raiz quadrada do valor
  - `exp()` : retorna a exponencial do valor
  - `log()` : retorna o logaritmo natual do valor
  - `log10()` : retorna o logaritmo na base 10 do valor
  - `greatest()` : retorna o maior valor dentre os valores das colunas. Análogo ao `max()`, mas entre colunas. 
  - `least()` : retorna o menor valor dentre os valores das colunas. Análogo ao `min()`, mas entre colunas

In [ ]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
(
    df_titles
    .withColumn('runtimeMinutes',f.col('runtimeMinutes').cast('int'))
    .withColumn('random_normal',f.randn(123))
    .withColumn('dummy_division',f.col('runtimeMinutes')/f.col('random_normal'))
    .withColumn('dummy_division',f.round(f.col('random_normal'),3))
    .withColumn('dummy_round',f.col('runtimeMinutes')*f.col('random_normal'))
    .withColumn('dummy_sum',f.col('runtimeMinutes')+f.col('random_normal'))
    .withColumn('dummy_sub',f.col('runtimeMinutes')-f.col('random_normal'))
    .withColumn('dummy_exp',f.col('runtimeMinutes')**f.col('random_normal'))
    .limit(5)
    .toPandas()
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,random_normal,dummy_division,dummy_round,dummy_sum,dummy_sub,dummy_exp
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",-0.992755,-0.993,-0.992755,0.007245,1.992755,1.000000
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",0.431839,0.432,2.159195,5.431839,4.568161,2.003745
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",0.250836,0.251,1.003345,4.250836,3.749164,1.415854
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",-0.104760,-0.105,-1.257121,11.895240,12.104760,0.770806
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",-1.301178,-1.301,-1.301178,-0.301178,2.301178,1.000000


**Strings**

  - `upper()` : retorna o string em letras maiúsculas
  - `lower()` : retorna o string em letras minúsculas
  - `initcap()` : retorna a primeira letra de cada palavra no string em letras maiúsculas
  - `trim()` : retira os espaços em branco do início e do final do string
  - `ltrim()/rtrim()` : retira os espaços em branco no início e do final do string, respectivamente
  - `lpad()/rpad()` : acrescenta um carctere no início e no final do srting, respectivamente, até que string tenha um determinado comprimento
  - `length()` : retorna o comprimento do stirng, em quantidade de caracteres
  - `split()` : quebra o string a partir de um padrão e retorna um array com os strings resultantes
  - `concat()` : concatena uma ou mais colunas de string
  - `concat_ws` : concatena uma ou mais colunas de string, com um separador entre elas
  - `regexp_extract()` : retorna um match no string a partir de um padrão regex
  - `regex_replace()` : substitui um match no string a partir de um padrão regex com outros caracteres
  - `substring()` : retornamos caracteres do sting que estão entre os índices especificados. Análogo a f.col().substring() 

In [ ]:
(
    df_titles
    .withColumn('primarytitle',f.upper(f.col('primaryTitle')))
    .withColumn('titleType',f.trim(f.initcap(f.col('titleType'))))
    .withColumn('genres_array',f.split(f.col('genres'),','))
    .withColumn('num_cost',f.substring(f.col('tconst'),3,7))
    .limit(5)
    .toPandas()
)

,tconst,titleType,primarytitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_array,num_cost
0,tt0000001,Short,CARMENCITA,Carmencita,0,1894,\N,1,"Documentary,Short","[Documentary, Short]",0000001
1,tt0000002,Short,LE CLOWN ET SES CHIENS,Le clown et ses chiens,0,1892,\N,5,"Animation,Short","[Animation, Short]",0000002
2,tt0000003,Short,PAUVRE PIERROT,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance","[Animation, Comedy, Romance]",0000003
3,tt0000004,Short,UN BON BOCK,Un bon bock,0,1892,\N,12,"Animation,Short","[Animation, Short]",0000004
4,tt0000005,Short,BLACKSMITH SCENE,Blacksmith Scene,0,1893,\N,1,"Comedy,Short","[Comedy, Short]",0000005


**Datas**

  - `add_months()` : retorna a data depois de adicionar "x" meses
  - `months_between()` : retorna a diferença entre duas datas em meses
  - `date_add()` : retorna a data depois de adicionar "x" dias
  - `date_sub()` : retorna a data depois de subtrair "x" dias
  - `next_day` : retorna o dia seguinte de alguma data
  - `datediff()` : retorna a diferença entre duas datas em dias
  - `current_date` : retorna a data atual
  - `dayofweek/dayofmonth()/dayofyear` : retorna o dia relativo à semana, ao mês e ao ano, respectivamente.
  - `weekofyear` : retorna a semana relativa ao ano
  - `second()/minute()/hour()` : retorna os segundos, os minutos e as horas de uma coluna de date-time, respectivamente
  - `month()/year()` : retorna o mês e o ano de uma coluna de data, respectivamente
  - `last_day()` : retorna o último dia do mês do qual a data considerada pertence
  - `to_date()` : transforma a coluna no tipo data (t.DateType())
  - `trunc()` : formata a data para a unidade especificada
    - `year` : "(ano)-01-01"
    - `month` : "(ano)-(mes)-01" 

In [ ]:
(
    df_titles
    .filter('titleType = "movie"')
    .withColumn('data_ano',f.to_date(f.col('startYear'),'yyyy'))
    .withColumn('mes',f.month(f.col('data_ano')))
    .withColumn('ano',f.dayofmonth(f.col('data_ano')))
    .withColumn('hoje',f.current_date())
    .withColumn('trunc',f.trunc(f.col('data_ano'),'week'))
    .withColumn('ultimo_dia_mes',f.last_day(f.col('data_ano')))
    .withColumn('idade_filme_dias',f.datediff(f.col('hoje'),f.col('data_ano')))
    .withColumn('idade_filme_meses',f.floor(f.months_between(f.col('hoje'),f.col('data_ano'))))
    .withColumn('idade_filme_anos',f.floor(f.col('idade_filme_dias')/365))
    .select('startYear','data_ano','mes','ano','hoje','trunc',
            'ultimo_dia_mes','idade_filme_dias','idade_filme_meses','idade_filme_anos')
    .show(5)
)

+---------+----------+---+---+----------+----------+--------------+----------------+-----------------+----------------+
|startYear|  data_ano|mes|ano|      hoje|     trunc|ultimo_dia_mes|idade_filme_dias|idade_filme_meses|idade_filme_anos|
+---------+----------+---+---+----------+----------+--------------+----------------+-----------------+----------------+
|     1905|1905-01-01|  1|  1|2022-05-12|1904-12-26|    1905-01-31|           42865|             1408|             117|
|     1907|1907-01-01|  1|  1|2022-05-12|1906-12-31|    1907-01-31|           42135|             1384|             115|
|     1907|1907-01-01|  1|  1|2022-05-12|1906-12-31|    1907-01-31|           42135|             1384|             115|
|     1908|1908-01-01|  1|  1|2022-05-12|1907-12-30|    1908-01-31|           41770|             1372|             114|
|     1908|1908-01-01|  1|  1|2022-05-12|1907-12-30|    1908-01-31|           41770|             1372|             114|
+---------+----------+---+---+----------

### **Arrays**

  - `array()` : constrói um array com as colunas selecionadas
  - `flatten()` : transforma um array de arrays em um único array
  - `explode()` : retorna uma nova linha para cada elemento no array
  - `size()` : retorna o número de elementos do array
  - `sort_array()` : ordena os elementos do array, de forma crescente ou decrescente
  - `reverse()` : reverte a ordem dos elementos de um array 
  - `array_distinct()` : remove elementos duplicados do array
  - `array_contains()` : verifica se o array contém o elemento especificado
  - `arrays_overlap()` : a partir de 2 colunas de arrays, verifica se elas tem algum elemento em comum, retornando True ou False
  - `array_union()` : a partir de 2 colunas de arrays, retorna o array com os elementos unidos das duas colunas, sem duplicatas
  - `array_except()` : a partir de 2 colunas de arrays, retorna um array com os elementos que estão em uma coluna mais não estão na outra, sem duplicatas
  - `array_intersect()` : a partir de 2 colunas de arrays, retorna um array com os elementos que estão nas duas colunas, sem duplicatas
  - `array_join()` : retorna um string após concaterna os elementos do array usando um delimitador especificado
  - `array_max()/array_min()` : retorna o máximo e o mínimo valor do array, respectivamente
  - `array_remove()` : remove elementos do array que são iguais a uma valor especificado  


In [ ]:
(
    df_titles
    .filter('titleType = "movie"')
    .withColumn('genres_array',f.split(f.col('genres'), ','))
    .withColumn('first_genre',f.col('genres_array')[0])
    .withColumn('genres_string',f.array_join(f.col('genres_array'),','))
    .withColumn('n_genres',f.size(f.col('genres_array')))
    .withColumn('genres_unico',f.explode(f.col('genres_array')))
    .select('genres_array','first_genre','genres_string','n_genres','genres_unico')
    .show(20)
)

+--------------------+-----------+-----------------+--------+------------+
|        genres_array|first_genre|    genres_string|n_genres|genres_unico|
+--------------------+-----------+-----------------+--------+------------+
|                [\N]|         \N|               \N|       1|          \N|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|[Adventure, Fantasy]|  Adventure|Adventure,Fantasy|       2|   Adventure|
|[Adventure, Fantasy]|  Adventure|Adventure,Fantasy|       2|     Fantasy|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|                [\N]|         \N|               \N|       1|          \N|
|                [\N]|   

**Nulos**

  - `drop()` : retira do DataFrame as linhas com nulos, com base no que foi passado para o argumento how
    - any (default) : retira todas as linhas com pelo menos um valor nulo nas colunas
    - all : somente retira as linhas com todos os valores nulos nas colunas
  - `fill()` : preenche os valores nulos no DataFrame com uma constante, passada pelo usuário
  - `replace()` : substitui o valor (não somente os valores nulos) por algum outro passado pelo usuário  

In [ ]:
(
    df_titles
    .replace('\\N', None, subset=['startYear','endYear'])
    .filter('endYear is null and startYear is not null')
    #.na.drop(subset=['startYear'])#'0',subset=['endYear'])
    .withColumn('coalesce_test',f.coalesce(f.col('startYear'),
                                           f.col('startYear'),
                                           f.lit('Sem ano')))
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|coalesce_test|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   null|             1|   Documentary,Short|         1894|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   null|             5|     Animation,Short|         1892|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|   null|             4|Animation,Comedy,...|         1892|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   null|            12|     Animation,Short|         1892|
|tt0000005|    short

# **Aula 4.1 -** Manipulando dados com Spark - Parte II

### **Agrupamento e Agregação**

O agurpamento dos DataFrames é feito por meio da função **groupby()**. Essa função deve ser sucedida pela função agregação `agg()`, de pivotação `pivot()` ou `count()`. A função **agg()** aplica uma função de agregação no DataFrame. Se
precedida por `groupby()`, realiza a agregação dentro dos grupos estabelecidos.
Algumas das funções de agregação mais utilizadas são : 

- `sum()` : retorna a soma dos valores da coluna;
- `sumDistinct()` : retorna a soma dos valores distintos da coluna;
- `max() / min()` : retorna o mínimo e o máximo da coluna, respectivamente.
- `avg() / mean()`: retorna a média dos valores da coluna;
- `percentile_approx()` : retorna o percentil da coluna, com aproximação. Para
trazer a mediana exata, usar : `percentile_approx(f.col(column),.5,lit(10000000))` 
- `stddev()` : retorna o desvio padrão dos valores da coluna
- `count()` : retorna a contagem de linhas
- `countDistinct` : retorna a contagem de valores distintos da coluna
- `first() / last()` : retorna o primeiro e o último valor da coluna no agrupamento, respectivamente. Interessante de ser utilizada em conjunto com o agrupamento `ignoreNulls=True`.
- `collect_list()`: retorna os valores do agrupamento em uma lista, com duplicações.
- `collect_set()` : retorna os valores do agrupamento em uma lista, sem duplicações (desordenado)

**obs:** O spark ignora os valores nulos para calcular as agregações, com exceção da função `count()`.
*texto em itálico*
---

A função **pivot** é utilizada para passar valores de uma linha para as colunas, realizando uma agregação. Deve ser sucedido por uma função de agregação utilizando `agg()`. Pode utilizar qualquer umas das funções de agregação anteriores. 

In [ ]:
df_titles.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
df_titles_subset = (
    df_titles
    .filter("cast(startYear as int) >= 2000")
    .sample(fraction = 0.5)
    .withColumn('genre', f.split('genres',',').getItem(0))
) 

In [ ]:
df_titles_subset.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|      genre|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+
|tt0015414|    movie|La tierra de los ...|La tierra de los ...|      0|     2000|     \N|            60|                  \N|         \N|
|tt0035423|    movie|      Kate & Leopold|      Kate & Leopold|      0|     2001|     \N|           118|Comedy,Fantasy,Ro...|     Comedy|
|tt0036177|    movie|  Muhomatsu no issho|  Muhomatsu no issho|      0|     2008|     \N|           100|    Action,Adventure|     Action|
|tt0056840|    short|            Aufsätze|            Aufsätze|      0|     2021|     \N|            10|               Short|      Short|
|tt0062336|    movie|The Tango of 

In [ ]:
(
    df_titles_subset
    .agg(f.countDistinct('genre').alias('distinct_genres'),)
    .show()
)

+---------------+
|distinct_genres|
+---------------+
|             28|
+---------------+



In [ ]:
(
    df_titles_subset
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .agg(f.sum('runtimeMinutes').alias('total_runtimeMinutes'),
         f.mean('runtimeMinutes').alias('mean_runtimeMinutes'),
         f.min('runtimeMinutes').alias('min_runtimeMinutes'),
         f.max('runtimeMinutes').alias('min_runtimeMinutes'),
         f.percentile_approx('runtimeMinutes', 0.5, f.lit(100000000)).alias('median_runtimeMinutes'),
         f.stddev('runtimeMinutes').alias('std_runtimeMinutes'),
        )
    .show()
)

+--------------------+-------------------+------------------+------------------+---------------------+------------------+
|total_runtimeMinutes|mean_runtimeMinutes|min_runtimeMinutes|min_runtimeMinutes|median_runtimeMinutes|std_runtimeMinutes|
+--------------------+-------------------+------------------+------------------+---------------------+------------------+
|            34124331| 40.584440482717525|                 0|             51420|                   26| 87.98492400220432|
+--------------------+-------------------+------------------+------------------+---------------------+------------------+



In [ ]:
(
  df_titles_subset
  .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int')).select('runtimeMinutes').describe().toPandas()
)

,summary,runtimeMinutes
0,count,840823
1,mean,40.584440482717525
2,stddev,87.98492400220432
3,min,0
4,max,51420


### **Agrupamentos**

In [ ]:
df_titles_subset.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|      genre|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+
|tt0015414|    movie|La tierra de los ...|La tierra de los ...|      0|     2000|     \N|            60|                  \N|         \N|
|tt0035423|    movie|      Kate & Leopold|      Kate & Leopold|      0|     2001|     \N|           118|Comedy,Fantasy,Ro...|     Comedy|
|tt0036177|    movie|  Muhomatsu no issho|  Muhomatsu no issho|      0|     2008|     \N|           100|    Action,Adventure|     Action|
|tt0056840|    short|            Aufsätze|            Aufsätze|      0|     2021|     \N|            10|               Short|      Short|
|tt0062336|    movie|The Tango of 

In [ ]:
(
    df_titles_subset
    .groupby('genre')
    .count()
    .orderBy(f.col('count').desc())
    .show()
)

+-----------+------+
|      genre| count|
+-----------+------+
|     Comedy|483222|
|      Drama|461251|
|Documentary|242567|
|         \N|182687|
|  Talk-Show|172527|
|       News|170164|
| Reality-TV|121215|
|     Action|113018|
|      Adult|110649|
|  Animation| 86924|
|      Music| 76637|
|      Crime| 75005|
|      Short| 72493|
|  Adventure| 60987|
|     Family| 59619|
|  Game-Show| 59381|
|      Sport| 39828|
|  Biography| 30268|
|     Horror| 30021|
|    Romance| 28419|
+-----------+------+
only showing top 20 rows



In [ ]:
(
    df_titles_subset
    .groupby('genre')
    .agg(f.mean('runtimeMinutes').alias('mean_runtimeMinutes'),)
    .orderBy(f.col('mean_runtimeMinutes').desc())
    .show()
)

+-----------+-------------------+
|      genre|mean_runtimeMinutes|
+-----------+-------------------+
|      Adult|  96.38374065115245|
|        War|  82.54285714285714|
|      Sport|  82.26542239685658|
|   Thriller|  77.75132802124834|
|    Western|  71.73099415204679|
|       News|  55.72532383419689|
|         \N| 53.155585520174256|
|      Crime|  49.59258324128136|
|  Biography|  49.48170964054363|
|  Talk-Show|  48.54595962713748|
|Documentary|  47.87597944822568|
|    Musical|  47.74240583232078|
|  Game-Show|  47.26210941450096|
|    Romance|  44.65932322876278|
|     Action|  43.02412671578986|
|    History|  42.76622825625795|
|     Family|  40.60426255753452|
|      Drama| 39.629701517105154|
|    Mystery| 37.997517994539585|
|  Adventure| 37.444982312418546|
+-----------+-------------------+
only showing top 20 rows



In [ ]:
(
    df_titles_subset
    .groupby('genre','startYear')
    .agg(f.mean('runtimeMinutes').alias('mean_runtimeMinutes'),)
    .orderBy('startYear',f.col('mean_runtimeMinutes').desc())
    .show()
)

+-----------+---------+-------------------+
|      genre|startYear|mean_runtimeMinutes|
+-----------+---------+-------------------+
|        War|     2000|              113.5|
|    Western|     2000|               95.0|
|      Adult|     2000|  94.67576075550892|
|   Thriller|     2000|   90.8051948051948|
|      Sport|     2000|  88.73684210526316|
|    Musical|     2000|  81.44186046511628|
|    History|     2000|  76.92307692307692|
|  Biography|     2000|  72.04102564102564|
|  Adventure|     2000|  70.90107526881721|
|    Mystery|     2000|  64.41379310344827|
|       News|     2000|  63.87619047619047|
|      Crime|     2000|  63.87202925045704|
|     Horror|     2000|  63.62809917355372|
|         \N|     2000|  57.84417549167927|
|      Drama|     2000|   53.5015197568389|
|    Romance|     2000|              52.04|
|Documentary|     2000| 51.834316037735846|
|     Comedy|     2000|               50.8|
|     Action|     2000|    49.921826625387|
|  Talk-Show|     2000| 49.17105

In [ ]:
(
    df_titles_subset
    .groupby('genre','startYear')
    .agg(f.mean('runtimeMinutes').alias('mean_runtimeMinutes'),)
    .orderBy(f.col('mean_runtimeMinutes').desc())
    .filter('startYear = 2021')
    .show()
)

+-----------+---------+-------------------+
|      genre|startYear|mean_runtimeMinutes|
+-----------+---------+-------------------+
|      Sport|     2021|  108.6083650190114|
|    Western|     2021|               90.3|
|   Thriller|     2021|   88.7081081081081|
|        War|     2021|  74.33333333333333|
|  Biography|     2021|  73.66771159874608|
|      Adult|     2021|  64.97372488408037|
|         \N|     2021|  64.70121381886088|
|       News|     2021|  62.99842271293375|
|  Game-Show|     2021| 53.718333333333334|
|     Action|     2021|  52.32767700409596|
|  Adventure|     2021|   51.4042904290429|
|      Crime|     2021| 49.656042496679945|
| Reality-TV|     2021|   48.9959100204499|
|    History|     2021|  47.26373626373626|
|Documentary|     2021| 45.165052631578945|
|    Mystery|     2021| 43.920731707317074|
|    Romance|     2021| 43.464454976303315|
|  Talk-Show|     2021| 42.887052341597794|
|      Drama|     2021| 41.683746246246244|
|     Family|     2021| 41.26244

In [ ]:
(
    df_titles_subset
    .groupby('genre')
    .agg(f.collect_list(f.col('titleType')).alias('title_type_list'))
    .orderBy('genre')
    .show()
)

+-----------+--------------------+
|      genre|     title_type_list|
+-----------+--------------------+
|     Action|[movie, tvSeries,...|
|      Adult|[video, movie, vi...|
|  Adventure|[tvMovie, movie, ...|
|  Animation|[short, short, sh...|
|  Biography|[movie, movie, mo...|
|     Comedy|[movie, movie, mo...|
|      Crime|[movie, movie, mo...|
|Documentary|[short, movie, mo...|
|      Drama|[movie, movie, mo...|
|     Family|[movie, movie, tv...|
|    Fantasy|[movie, movie, vi...|
|  Game-Show|[tvSeries, tvSeri...|
|    History|[movie, tvSeries,...|
|     Horror|[movie, movie, mo...|
|      Music|[tvSeries, video,...|
|    Musical|[movie, video, tv...|
|    Mystery|[movie, movie, mo...|
|       News|[tvSeries, tvSeri...|
| Reality-TV|[tvSeries, tvSpec...|
|    Romance|[movie, movie, mo...|
+-----------+--------------------+
only showing top 20 rows



In [ ]:
(
    df_titles_subset
    .groupby('genre')
    .agg(f.collect_list(f.col('titleType')).alias('title_type_list'),
         f.count(f.lit(1)).alias('count')
         )
    .withColumn('n',f.size(f.col('title_type_list')))
    .orderBy('genre')
    .show()
)

+-----------+--------------------+------+------+
|      genre|     title_type_list| count|     n|
+-----------+--------------------+------+------+
|     Action|[movie, tvSeries,...|113018|113018|
|      Adult|[video, movie, vi...|110649|110649|
|  Adventure|[tvMovie, movie, ...| 60987| 60987|
|  Animation|[short, short, sh...| 86924| 86924|
|  Biography|[movie, movie, mo...| 30268| 30268|
|     Comedy|[movie, movie, mo...|483222|483222|
|      Crime|[movie, movie, mo...| 75005| 75005|
|Documentary|[short, movie, mo...|242567|242567|
|      Drama|[movie, movie, mo...|461251|461251|
|     Family|[movie, movie, tv...| 59619| 59619|
|    Fantasy|[movie, movie, vi...| 14081| 14081|
|  Game-Show|[tvSeries, tvSeri...| 59381| 59381|
|    History|[movie, tvSeries,...|  9690|  9690|
|     Horror|[movie, movie, mo...| 30021| 30021|
|      Music|[tvSeries, video,...| 76637| 76637|
|    Musical|[movie, video, tv...|  5120|  5120|
|    Mystery|[movie, movie, mo...|  8573|  8573|
|       News|[tvSeri

In [ ]:
(
    df_titles_subset
    .groupby('genre')
    .agg(f.collect_set(f.col('titleType')).alias('title_type_list'),
         f.countDistinct(f.col('titleType')).alias('n_distinct')
         )
    .withColumn('n',f.size(f.col('title_type_list')))
    .orderBy('genre')
    .show()
)

+-----------+--------------------+----------+---+
|      genre|     title_type_list|n_distinct|  n|
+-----------+--------------------+----------+---+
|     Action|[tvSpecial, video...|        10| 10|
|      Adult|[tvSpecial, video...|        10| 10|
|  Adventure|[tvSpecial, video...|        10| 10|
|  Animation|[tvSpecial, video...|        10| 10|
|  Biography|[tvSpecial, video...|        10| 10|
|     Comedy|[tvSpecial, video...|        10| 10|
|      Crime|[tvSpecial, video...|        10| 10|
|Documentary|[tvSpecial, video...|        10| 10|
|      Drama|[tvSpecial, video...|        10| 10|
|     Family|[tvSpecial, video...|        10| 10|
|    Fantasy|[tvSpecial, video...|        10| 10|
|  Game-Show|[tvSpecial, video...|         9|  9|
|    History|[tvSpecial, video...|        10| 10|
|     Horror|[tvSpecial, video...|        10| 10|
|      Music|[tvSpecial, video...|        10| 10|
|    Musical|[tvSpecial, video...|        10| 10|
|    Mystery|[tvSpecial, video...|        10| 10|


In [ ]:
(
    df_titles_subset
    .groupby('genre')
    .agg(f.collect_list(f.col('titleType')).alias('title_type_list'),
         f.count(f.lit(1)).alias('count')
         )
    .withColumn('tipos_filmes',f.explode(f.col('title_type_list')))
    .select('genre','tipos_filmes')
    .show()
)

+-----+------------+
|genre|tipos_filmes|
+-----+------------+
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
|Crime|       movie|
+-----+------------+
only showing top 20 rows



### **Pivotação**

In [ ]:
(
    df_titles_subset
    .drop('genre')
    .withColumn('genres',f.explode(f.split(f.col('genres'), ',')))
    .groupby('tconst','primaryTitle')
    .pivot('genres')
    .count()
    .na.fill(0)
    .orderBy('tconst')
    .show()
)

+---------+--------------------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
|   tconst|        primaryTitle|Action|Adult|Adventure|Animation|Biography|Comedy|Crime|Documentary|Drama|Family|Fantasy|Game-Show|History|Horror|Music|Musical|Mystery|News|Reality-TV|Romance|Sci-Fi|Short|Sport|Talk-Show|Thriller|War|Western| \N|
+---------+--------------------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
|tt0015414|La tierra de los ...|     0|    0|        0|        0|        0|     0|    0|          0|    0|     0|      0|        0|      0|     0|    0|      0|      0|   0|         0|      0|     0|    0|    0|        0|       0|  0|      0|  1|
|tt0035423| 

In [ ]:
(
    df_titles_subset
    .drop('genre')
    .withColumn('genres',f.explode(f.split(f.col('genres'), ',')))
    .groupby('startYear')
    .pivot('genres')
    .agg(f.mean('runtimeMinutes'))
    .na.fill(0)
    .orderBy('startYear')
    .show()
)

+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|startYear|            Action|             Adult|         Adventure|         Animation|         Biography|            Comedy|             Crime|       Documentary|             Drama|            Family|           Fantasy|         Game-Show|           History|            Horror|             Music|           Musical|           Mystery|              News|        Reality-TV|           Romance|            Sci-Fi|             Short|             Sport|        

# **Aula 4.2**- Manipulando dados com Spark - Parte II

### **Window Functions**

Window functions são funções que realizam cálculos similares à uma agregação, mas que não resultam em um DataFrame agregado. Ao invés disso, os resultados são colocados em uma nova coluna, segundo a partição (ou agrupamento) especificado. Exemplos mais comuns:
- `row_number`
- `rank() / dense_rank() / percent_rank()`
- `lag()`
- `cume_dist()`
- `collect_list() / collect_set()`

Para usar as funções dessa forma, devemos criar uma janela (window) da 
seguinte forma:

  `from pyspark.sql.window import window`

  `w = windowm.partitionBy({columns}).orderBy({columns}).rowsBetween({lower},{upper})`
  
- `partitionBy()` : agrupamentos em que os cálculos serão realizados. É análogo
ao `groupBy()`.
- `orderBy()` : algumas funções como `row_number()` e `lag()` dependem da 
ordenação das linhas do agrupamento. Essa função é usada para especificar essa
ordem. 
- `rowsBetween()` : esse método é usado para especificar janelas deslizantes. A partir dele é possível definir um intervalo de linhas, relativas à linha atual, 
em que a função vai ser aplicada. Caso isso não seja especificado, as operações
são realizadas em todo o grupo. Muito útil para construir **médias móveis**. Os seguintes objetos ajudam na construção desse intervalo.
- `Window.currentRow` : define a linha para qual o valre está sendo calculado como um dos limites de cálculo.
- `Window.unboundedPreceding` : define que não há limites para as linhas anteriores à linhas para qual o valor está sendo calculado, isto é, a função irá considerar todas as linhas do grupo que já passaram. Deve ser usado no primeiro argumento (start). 
- `Window.unboundedFollowing` : define que não há limites para as linhas posteriores à linha para qual o valore está sendo calculado, isto é, a função irá considerar todas as linhas do grupo que ainda não passaram. Deve ser usado
no segundo argumento (end). 

Depois disso, basta utilizar a função `over()` para indicar que aquela  função deve ser realizada na janela. Exemplo:

  `df.withColumn('rn',f.row_number().over(w))`

In [ ]:
from pyspark.sql.window import Window

In [ ]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
df_titles_subset = (
    df_titles
    .filter("cast(startYear as int) >= 2000")
    .sample(fraction = 0.5)
    .withColumn('genre', f.split('genres',',').getItem(0))
) 

In [ ]:
w = Window.partitionBy('genre').orderBy(f.desc('startYear'))

(
    df_titles_subset
    .withColumn('genre',f.split('genres',',').getItem(0))
    .withColumn('startYear', f.col('startYear').cast('int'))
    .withColumn('rn', f.rank().over(w))
    .filter('startYear <= 2021')
    .show()
)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+---+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|    genre| rn|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+---+
|tt10075380|    movie| Realm of Terracotta| Realm of Terracotta|      0|     2021|     \N|           111|Adventure,Animati...|Adventure|284|
|tt10084832|    movie|               Nomad|               Nomad|      0|     2021|     \N|            \N|Adventure,Drama,R...|Adventure|284|
|tt10121862|tvEpisode|        Episode #1.1|        Episode #1.1|      0|     2021|     \N|            \N|    Adventure,Comedy|Adventure|284|
|tt10155524|tvEpisode|Welcome to Monste...|Welcome to Monste...|      0|     2021|     \N|            \N|Adventure,Animati...|Adventure|284|
|tt10155932| 

In [ ]:
w = Window.partitionBy('genre').orderBy(f.desc('startYear'))

(
    df_titles_subset
    .withColumn('genre',f.split('genres',',').getItem(0))
    .withColumn('startYear', f.col('startYear').cast('int'))
    .withColumn('rn', f.percent_rank().over(w))
    .filter('startYear <= 2021')
    .show()
)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|    genre|                  rn|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+--------------------+
|tt10075380|    movie| Realm of Terracotta| Realm of Terracotta|      0|     2021|     \N|           111|Adventure,Animati...|Adventure|0.004708505257553574|
|tt10084832|    movie|               Nomad|               Nomad|      0|     2021|     \N|            \N|Adventure,Drama,R...|Adventure|0.004708505257553574|
|tt10121862|tvEpisode|        Episode #1.1|        Episode #1.1|      0|     2021|     \N|            \N|    Adventure,Comedy|Adventure|0.004708505257553574|
|tt10155524|tvEpisode|Welcome to Monste...|Welcome t

In [ ]:
w = Window.partitionBy('titleType','startYear').orderBy('titleType','startYear')

(
    df_titles_subset
    .withColumn('genre',f.split('genres',',').getItem(0))
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .withColumn('total_minutes', f.sum(f.col('runtimeMinutes')).over(w))
    .withColumn('mean_minutes', f.mean(f.col('runtimeMinutes')).over(w))
    .withColumn('relative_minutes', f.col('runtimeMinutes') / f.col('total_minutes'))
    .withColumn('lag_runtime', f.lag('runtimeMinutes').over(w))
    .filter('runtimeMinutes is not null')
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-------------+-----------------+--------------------+-----------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|      genre|total_minutes|     mean_minutes|    relative_minutes|lag_runtime|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-------------+-----------------+--------------------+-----------+
|tt0102362|    movie|              Istota|              Istota|      0|     2000|     \N|            80|       Drama,Romance|      Drama|       177555|91.33487654320987|4.505646137816451E-4|       null|
|tt0115937|    movie|         Consequence|         Consequence|      0|     2000|     \N|            91|               Drama|      Drama|       177555|91.33487654320987|5.125172481766213E-

In [ ]:
w = Window.partitionBy('titleType').orderBy('startYear').rowsBetween(-2, Window.currentRow)
(
    df_titles_subset
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .groupby('titleType','startYear')
    .agg(f.expr('mean(runtimeMinutes) as media_minutes'))
    .orderBy('titleType','startYear')
    .withColumn('media_movel_3anos', f.round(f.mean('media_minutes').over(w),3))
    .show()
)

+---------+---------+-----------------+-----------------+
|titleType|startYear|    media_minutes|media_movel_3anos|
+---------+---------+-----------------+-----------------+
|    movie|     2000|91.33487654320987|           91.335|
|    movie|     2001|91.67862481315396|           91.507|
|    movie|     2002|90.93029739776952|           91.315|
|    movie|     2003|90.23226072607261|           90.947|
|    movie|     2004|89.11744583808438|           90.093|
|    movie|     2005|89.72853368560105|           89.693|
|    movie|     2006|89.09390243902439|           89.313|
|    movie|     2007|89.61379700310998|           89.479|
|    movie|     2008|88.16112594030575|           88.956|
|    movie|     2009|87.24494623655914|            88.34|
|    movie|     2010|87.58061985092193|           87.662|
|    movie|     2011|90.25551641016132|            88.36|
|    movie|     2012| 87.5018398458034|           88.446|
|    movie|     2013|88.70692194403534|           88.821|
|    movie|   

In [ ]:
a = (90.35232168501676 + 91.40670289855072 + 90.03873684210527)/3.
a

90.59925380855759

In [ ]:
# Aqui temos uma média incremental. Ela sempre pega todos os valores para trás. Diferente do 
# caso anterior. Que sempre pegava duas 3 para trás. 
w = Window.partitionBy('titleType').orderBy('startYear').rowsBetween(Window.unboundedPreceding, Window.currentRow)
(
    df_titles_subset
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .groupby('titleType','startYear')
    .agg(f.expr('mean(runtimeMinutes) as media_minutes'))
    .orderBy('titleType','startYear')
    .withColumn('media_movel_3anos', f.round(f.mean('media_minutes').over(w),3))
    .show()
)

+---------+---------+-----------------+-----------------+
|titleType|startYear|    media_minutes|media_movel_3anos|
+---------+---------+-----------------+-----------------+
|    movie|     2000|91.33487654320987|           91.335|
|    movie|     2001|91.67862481315396|           91.507|
|    movie|     2002|90.93029739776952|           91.315|
|    movie|     2003|90.23226072607261|           91.044|
|    movie|     2004|89.11744583808438|           90.659|
|    movie|     2005|89.72853368560105|           90.504|
|    movie|     2006|89.09390243902439|           90.302|
|    movie|     2007|89.61379700310998|           90.216|
|    movie|     2008|88.16112594030575|           89.988|
|    movie|     2009|87.24494623655914|           89.714|
|    movie|     2010|87.58061985092193|            89.52|
|    movie|     2011|90.25551641016132|           89.581|
|    movie|     2012| 87.5018398458034|           89.421|
|    movie|     2013|88.70692194403534|            89.37|
|    movie|   

In [ ]:
(91.13144058885383 + 90.35232168501676 + 91.40670289855072 + 90.03873684210527+ 89.91844369622147)/5.

90.5695291421496

In [ ]:
w = Window.partitionBy('titleType','startYear')
(
    df_titles_subset
    .withColumn('lista_titulos', f.collect_set(f.col('tconst')).over(w))
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|      genre|       lista_titulos|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+--------------------+
|tt0102362|    movie|              Istota|              Istota|      0|     2000|     \N|            80|       Drama,Romance|      Drama|[tt0179749, tt024...|
|tt0107706|    movie|             Nothing|             Nothing|      0|     2000|     \N|            \N|                  \N|         \N|[tt0179749, tt024...|
|tt0113086|    movie|Florentino y el d...|Florentino y el d...|      0|     2000|     \N|            \N|               Drama|      Drama|[tt0179749, tt024...|
|tt0115937|    movie|         Consequence|    

In [ ]:
#w = Window.partitionBy('titleType','startYear')
#(
#    df_titles_subset
#    .withColumn('lista_titulos', f.collect_set(f.col('tconst')).over(w))
#    .withColumn('titulos_distintos', f.size(f.col('lista_titulos')))
#    .select('titleType','startYear','titulos_distintos')
#    .orderBy(f.col('titulos_distintos').desc())
#    .show()
#)
# It takes very long time

In [ ]:
#(
#    df_titles_subset
#    .groupby('titleType','startYear')
#    .agg(f.countDistinct(f.col('tconst')).alias('titulos_distintos'))
#    .orderBy(f.col('titulos_distintos').desc())
#    .show()
#)

### **Usando Windows para evitar joins**

**Objetivo :** Título mais recente por gênero

Caminho natual :

  `df1 = df_titles.goupby('genre').agg(f.max(f.col('startYear').alias('startYear')))`
  
  `df2 = df_titles.join(df1,['genre','startYear'])`

**Alternativa :**

In [ ]:
#w = Window.partitionBy('genre')
#(
#    df_titles_subset
#    .withColumn('max_data', f.max(f.col('startYear')).over(w))
#    .filter('startYear = max_data')
#    .show()
#)

# **Aula 4.3**- Manipulando dados com Spark - Parte II

### Tipos de Joins

Os joins no pyspark são especificados pela função `join()`, da seguinte forma:

`df1.join(df2,{key_columns},{join_type})`

- `key_columns` : colunas que vão ser utilizadas para fazer a junção das tabelas. Pode ser especifida como
  - Uma única string -> só uma coluna é chave, mesmos nomes nas duas tabelas. 
  - Uma lista de string ou de colunas (col()) -> mais de uma coluna é chave, mesmos nomes nas duas tabelas.
  - Com nomes diferentes, é necessário fazer uma especificação do tipo: 
  `f.col(column1) == f.col(column2)`. Caso existam mais de uma coluna como chave, essas especificações devem ser colocadas em uma lista. 
- `join_type`: o tipo de join a ser realizado. As opções são: 
  - `inner (default)`:: INNER JOIN do SQL
  - `outer / full / fullouter / full_outer`:: FULL_OUTER JOIN so SQL
  - `left / left_outer / left_outer / `:: LEFT JOIN do SQL
  - `right / rightouter / right_outer`:: RIGHT JOIN do SQL 
  - `semi / leftsemi / left_semi`:: realiza um LEFT JOIN do SQL e retorna somente as colunas do DataFrame da esquerda que também estão no DataFrame da Direita. 
  - `anti / leftanti / left_anti`:: realiza um LEFT JOIN do SQL e retorna somente as colunas do DataFrame da esquerda que não estão no DataFrame da Direita. 

In [ ]:
df_titles.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
imdb_path = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_ratings.tsv'

In [ ]:
options_dict = {
    'sep' : '\t' , 
    'header' : 'True'
}

df_ratings = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .load(imdb_path)
)

df_ratings.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1879|
|tt0000002|          5.9|     248|
|tt0000003|          6.5|    1652|
|tt0000004|          5.8|     161|
|tt0000005|          6.2|    2476|
+---------+-------------+--------+
only showing top 5 rows



In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst')
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|   Documentary,Short|          5.4|    2017|
|tt0000015|    short| Autour d'une cabine| Autour d'une cabine|      0|     1894|     \N|             2|     Animation,Short|          6.2|     994|
|tt0000019|    short|    The Clown Barber|    The Clown Barber|      0|     1898|     \N|            \N|        Comedy,Short|          5.2|      30|
|tt0000051|    short|The Bohemian Enca...|Campement de bohé...|      0|     1896|     \N|            \N|  

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst')
    .count()
)

1227178

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst', 'left')
    .filter('averageRating is null')
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|tt0000195|    short| Les farces de Jocko| Les farces de Jocko|      0|     1898|     \N|            \N|        Comedy,Short|         null|    null|
|tt0000212|    short|Saida do Paquete ...|Saida do Paquete ...|      0|     1898|     \N|            \N|               Short|         null|    null|
|tt0000221|    short|Aspectos da Praia...|Aspectos da Praia...|      0|     1899|     \N|            \N|   Documentary,Short|         null|    null|
|tt0000267|    short|           L'angélus|           L'angélus|      0|     1900|     \N|            \N|  

In [ ]:
(
    df_titles
    .withColumnRenamed('tconst','id_title')
    .join(df_ratings, f.col('tconst') == f.col('id_title'))
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+-------------+--------+
| id_title|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|   tconst|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+-------------+--------+
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|   Documentary,Short|tt0000008|          5.4|    2017|
|tt0000015|    short| Autour d'une cabine| Autour d'une cabine|      0|     1894|     \N|             2|     Animation,Short|tt0000015|          6.2|     994|
|tt0000019|    short|    The Clown Barber|    The Clown Barber|      0|     1898|     \N|            \N|        Comedy,Short|tt0000019|          5.2|      30|
|tt0000051|    short|The Bohemian Enca...|Camp

In [ ]:
(
    df_titles
    .withColumnRenamed('tconst','id_title')
    .join(df_ratings, f.col('tconst') == f.col('id_title'))
    .withColumn('averageRating', f.expr('averageRating + 1'))
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+-------------+--------+
| id_title|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|   tconst|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+---------+-------------+--------+
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|   Documentary,Short|tt0000008|          6.4|    2017|
|tt0000015|    short| Autour d'une cabine| Autour d'une cabine|      0|     1894|     \N|             2|     Animation,Short|tt0000015|          7.2|     994|
|tt0000019|    short|    The Clown Barber|    The Clown Barber|      0|     1898|     \N|            \N|        Comedy,Short|tt0000019|          6.2|      30|
|tt0000051|    short|The Bohemian Enca...|Camp

### Utilizando semi e anti join

In [ ]:
df_ratings.select('tconst').distinct().count()

1242221

In [ ]:
df_titles.select('tconst').distinct().count()

8328316

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst','inner')
    .distinct()
    .count()
)

1227178

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst','semi')
    .count()
)

1227178

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst','left_anti')
    .count()
)

In [ ]:
1227178 + 7101138 == df_titles.count()

### **Union** 

Existem três formas de unir DataFrames no pyspark:

- `union() / unionAll()`: empilha os Dataframes, preservando linhas duplicadas. as colunas são unidas por posição, e por isso a ordem delas deve ser a mesma entre os dois DF's. 
- `unionByname()`: empilha os DataFrames, preservando linhas duplicadas. As colunas são unidas por nome, e por tanto não precisam estar ordenadas da mesma forma.  

In [ ]:
df1 = df_titles.sample(fraction = .5)
df2 = df_titles.join(df1, ['tconst'], 'anti')

In [ ]:
df1.count()

4166563

In [ ]:
df2.count()

4161753

In [ ]:
df1.union(df2).count()

8328316

In [ ]:
df3 = df_titles.sample(fraction = 0.05)

In [ ]:
df3.count()

417468

In [ ]:
df3.union(df3).count()

834936

In [ ]:
df2 = df2.select(df2.columns[::-1])

In [ ]:
df1.union(df2).filter('genres rlike "[0-9]"').show()

+--------------------+---------+------------+-------------+-------+--------------------+--------------------+--------------+---------+
|              tconst|titleType|primaryTitle|originalTitle|isAdult|           startYear|             endYear|runtimeMinutes|   genres|
+--------------------+---------+------------+-------------+-------+--------------------+--------------------+--------------+---------+
|       Short,Western|       \N|          \N|         1910|      0|A Cowboy's Vindic...|A Cowboy's Vindic...|         short|tt0001170|
|      Crime,Thriller|       48|          \N|         1912|      0|Zigomar contre Ni...|Zigomar contre Ni...|         movie|tt0002588|
|                  \N|       \N|          \N|         1913|      0|             Zu spät|             Zu spät|         movie|tt0002591|
|     Adventure,Short|       \N|          \N|         1913|      0|The Battle for Fr...|The Battle for Fr...|         short|tt0002667|
|Comedy,Romance,Short|       \N|          \N|         1

In [ ]:
df1.unionByName(df2).filter('genres rlike "[0-9]"').show()

+------+---------+------------+-------------+-------+---------+-------+--------------+------+
|tconst|titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|
+------+---------+------------+-------------+-------+---------+-------+--------------+------+
+------+---------+------------+-------------+-------+---------+-------+--------------+------+



# **Aula 4.4**- Manipulando dados com Spark - Parte II

## User Defined Functions (UDFs)

Em algumas situações é necessário criar/alterar um coluna utilizando uma operação não implementada na biblioteca padrão. Para isso, é possível utilizar funções definidas pelo usuário (UDFs) por meio da função udf().

**Importante:** As udfs não são otimizadas para serem executadas em paralelo, e por isso podem representar um gargalo na aplicação. 

In [9]:
pip install unidecode

     |████████████████████████████████| 235 kB 13.4 MB/s 


In [10]:
from unidecode import unidecode
from pyspark.sql.types import StringType

In [11]:
unidecode('àáãçéõü')

'aaaceou'

In [14]:
def unidecode_function(string) :
    if not string :
      return None
    else :
      return unidecode(string)

unidecode_udf = f.udf(unidecode_function, returnType=StringType())

In [16]:
(
    df_titles
    .filter(f.col('primaryTitle').rlike('à|á|ã|ç|é|õ|ü|ó'))
    .withColumn('cleaned_string', unidecode_udf(f.col('primaryTitle')))
    .select('primaryTitle','cleaned_string')
    .toPandas()
    #.show(5)
)

,primaryTitle,cleaned_string
0,Arrivée d'un train gare de Vincennes,Arrivee d'un train gare de Vincennes
1,Batteuse à vapeur,Batteuse a vapeur
2,Bébé et fillettes,Bebe et fillettes
3,A Chegada do Comboio Inaugural à Estação Centr...,A Chegada do Comboio Inaugural a Estacao Centr...
4,Cortège de tzar allant à Versailles,Cortege de tzar allant a Versailles
...,...,...
190780,De la ilusión al desconcierto: cine colombiano...,De la ilusion al desconcierto: cine colombiano...
190781,La face cachée du mentalisme,La face cachee du mentalisme
190782,Développement personnel: pensez positif,Developpement personnel: pensez positif
190783,De Volta à Era Disco Music,De Volta a Era Disco Music


In [17]:
del unidecode_udf

In [21]:
@f.udf(returnType=StringType()) 
def unidecode_udf(string) :
    if not string :
      return None
    else :
      return unidecode(string)

In [23]:
(
    df_titles
    .filter(f.col('primaryTitle').rlike('à|á|ã|ç|é|õ|ü|ó'))
    .withColumn('cleaned_string', unidecode_udf(f.col('primaryTitle')))
    .select('primaryTitle','cleaned_string')
    .limit(10)
    .toPandas()
    #.show(5)
)

,primaryTitle,cleaned_string
0,Arrivée d'un train gare de Vincennes,Arrivee d'un train gare de Vincennes
1,Batteuse à vapeur,Batteuse a vapeur
2,Bébé et fillettes,Bebe et fillettes
3,A Chegada do Comboio Inaugural à Estação Centr...,A Chegada do Comboio Inaugural a Estacao Centr...
4,Cortège de tzar allant à Versailles,Cortege de tzar allant a Versailles
5,Libération des territoriaux,Liberation des territoriaux
6,Marée montante sur Brise-Larmes,Maree montante sur Brise-Larmes
7,"Place de l'Opéra, First View","Place de l'Opera, First View"
8,"Place de l'Opéra, Second View","Place de l'Opera, Second View"
9,Place du théâtre français,Place du theatre francais


## **Criando Métodos Customizados**

Em algumas situações, é interessante que realizemos uma operação sobre um DataFrame que não está implementada. Além disso, pode ser que seja necessário
utilizar esse operação de forma encadeada. 

Para resolver esse problema, podemos utilizar o método `.transform()`. Funciona da seguinte maneira : 

1) Definir uma função python da seguinte forma: 

```
  def f(args):
    def _(df):
      {operações sob o DataFrame}
      return_df
    return _
```
2) Depois de definida a função, ela pode ser chamada da seguinte forma:

    df.transform(f(args))

In [27]:
def renamer(dict):
  def _(df):
    for c,n in dict.items():
      df = df.withColumnRenamed(c,n)
    return df
  return _

In [28]:
df_titles.limit(5).show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [29]:
rename_dict = {
    "tconst" : "id_title",
    "titleType" : "tipo_title",
    "primaryTitle" : "nome_primario",
    "originalTitle" : "nome_original",
    "isAdult" : "idc_adult_title",
    "startYear" : "ano_lancamento",
    "endYear" : "ano_encerramento",
    "runtimeMinutes" : "duracao_minutos",
    "genres" : "generos",
}

(
    df_titles
    .transform(renamer(rename_dict))
    .show()
)

+---------+----------+--------------------+--------------------+---------------+--------------+----------------+---------------+--------------------+
| id_title|tipo_title|       nome_primario|       nome_original|idc_adult_title|ano_lancamento|ano_encerramento|duracao_minutos|             generos|
+---------+----------+--------------------+--------------------+---------------+--------------+----------------+---------------+--------------------+
|tt0000001|     short|          Carmencita|          Carmencita|              0|          1894|              \N|              1|   Documentary,Short|
|tt0000002|     short|Le clown et ses c...|Le clown et ses c...|              0|          1892|              \N|              5|     Animation,Short|
|tt0000003|     short|      Pauvre Pierrot|      Pauvre Pierrot|              0|          1892|              \N|              4|Animation,Comedy,...|
|tt0000004|     short|         Un bon bock|         Un bon bock|              0|          1892|     

In [31]:
# metódo mais antigo
from pyspark.sql import dataframe

def transform(self, f):
  return f(self)

df_titles.transform = transform